# LSTM multichannel single class classification

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from natsort import natsorted
import torch
import torch.nn as nn
from torch.nn import functional as F

from tensorboardX import SummaryWriter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import sys
sys.path.insert(0, "../")
from data_preparation.prepare_data import import_data
from model import make_loaders, eval_batch

### Data loading
It is necessary to significanlty reduce the initial sequence length of 6000 for LSTM network. The most challenging target to classify is `1` for which the model is able to learn only for a sequence length of 50 combined with an attention model

In [2]:
data_dir = '<path-to-dataset>'

channels = ['CP', 'FS1', 'PS1', 'PS2', 'PS3', 'PS4', 'PS5', 'SE', 'VS1']

targets = [1]
gpu = False
in_len = 50

print('Preparing data')
data = import_data(data_dir, in_len)
train_loader, test_loader = make_loaders(data, channels, targets)

Preparing data


In [3]:
labels = []
for i in range(len(train_loader.dataset)):
    labels.append(train_loader.dataset.__getitem__(i)['label'].item())
    
set(labels)

{0, 1, 2, 3}

### Model
M.Luong et al, Effective Approaches to Attention-based Neural Machine Translation, 2015, arXiv:1508.04025

Implementation from:
https://github.com/prakashpandey9/Text-Classification-Pytorch
https://github.com/spro/practical-pytorch/tree/master/seq2seq-translation

In [4]:
class Model(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=2):
        super(Model, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        
        self.fc_block = nn.Sequential(
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.BatchNorm1d(self.hidden_dim),
            nn.ReLU(inplace=True)
        ) 

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)
        
        self.concat = nn.Linear(self.hidden_dim * 2, self.hidden_dim)
        
#     def attention(self, lstm_output, hidden):
#     https://github.com/prakashpandey9/Text-Classification-Pytorch
        
#         hidden = hidden.squeeze(0)
#         lstm_output = lstm_output.permute(1,0,2)

#         score = torch.bmm(lstm_output, hidden.unsqueeze(2))
#         attn_weights = F.softmax(score, 1) # eq.7 
#         context = torch.bmm(lstm_output.transpose(1, 2), attn_weights).squeeze(2)
        
#         return context
    
    def attention(self, lstm_output, hidden):
        
        hidden = hidden.squeeze(0)
        lstm_output = lstm_output.permute(1,0,2)

        score = torch.bmm(lstm_output, hidden.unsqueeze(2))
        attn_weights = F.softmax(score, 1) # eq.7 
        context = torch.bmm(lstm_output.transpose(1, 2), attn_weights).squeeze(2)
        
        concat_input = torch.cat((hidden, context), 1)
        concat_output = torch.tanh(self.concat(concat_input)) # eq. 5
        
        return concat_output

    def forward(self, input):
        
        input = input.permute(2,0,1)
        lstm_out, (h,c) = self.lstm(input)
        
        out = self.attention(lstm_out, h[-1])
#         out = self.fc_block(lstm_out[-1])
        out = self.linear(out)
        
        return out

### Train Model

In [5]:
model = Model(9, 20, output_dim=4, num_layers=1)
model.train()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 10
for epoch in range(epochs):
        
    print('Starting epoch {}/{}.'.format(epoch + 1, epochs))
    epoch_loss = 0
    correct = 0

    for i, sample_batch in enumerate(train_loader):
        
        sequences = sample_batch['sequence']
        true_labels = sample_batch['label'].squeeze()

        pred_labels = model(sequences)
        
        _, predicted = torch.max(pred_labels, 1)
        correct += (predicted == true_labels).float().sum()

        loss = criterion(pred_labels, true_labels)
        epoch_loss += loss.item()
        
        if i%10 == 0:
            print(f'epoch = {epoch+1:d}, iteration = {i:d}/{len(train_loader):d}, loss = {loss.item():.5f}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1} finished! Loss: {epoch_loss/i}. Accuracy: {correct.item()/len(train_loader.dataset)}')

Starting epoch 1/10.
epoch = 1, iteration = 0/496, loss = 1.35050
epoch = 1, iteration = 10/496, loss = 1.32014
epoch = 1, iteration = 20/496, loss = 1.21037
epoch = 1, iteration = 30/496, loss = 1.41937
epoch = 1, iteration = 40/496, loss = 1.42623
epoch = 1, iteration = 50/496, loss = 1.49742
epoch = 1, iteration = 60/496, loss = 0.96758
epoch = 1, iteration = 70/496, loss = 1.40615
epoch = 1, iteration = 80/496, loss = 1.05095
epoch = 1, iteration = 90/496, loss = 1.03292
epoch = 1, iteration = 100/496, loss = 1.24753
epoch = 1, iteration = 110/496, loss = 1.02006
epoch = 1, iteration = 120/496, loss = 1.50157
epoch = 1, iteration = 130/496, loss = 0.99592
epoch = 1, iteration = 140/496, loss = 1.04547
epoch = 1, iteration = 150/496, loss = 1.58921
epoch = 1, iteration = 160/496, loss = 1.89748
epoch = 1, iteration = 170/496, loss = 1.52127
epoch = 1, iteration = 180/496, loss = 0.97000
epoch = 1, iteration = 190/496, loss = 0.94962
epoch = 1, iteration = 200/496, loss = 1.19137
epo

epoch = 4, iteration = 200/496, loss = 0.43310
epoch = 4, iteration = 210/496, loss = 0.87773
epoch = 4, iteration = 220/496, loss = 1.00469
epoch = 4, iteration = 230/496, loss = 0.60913
epoch = 4, iteration = 240/496, loss = 0.69908
epoch = 4, iteration = 250/496, loss = 1.12664
epoch = 4, iteration = 260/496, loss = 0.30037
epoch = 4, iteration = 270/496, loss = 1.23457
epoch = 4, iteration = 280/496, loss = 0.61078
epoch = 4, iteration = 290/496, loss = 1.40244
epoch = 4, iteration = 300/496, loss = 0.80328
epoch = 4, iteration = 310/496, loss = 1.64157
epoch = 4, iteration = 320/496, loss = 0.69980
epoch = 4, iteration = 330/496, loss = 1.59572
epoch = 4, iteration = 340/496, loss = 0.97596
epoch = 4, iteration = 350/496, loss = 1.19923
epoch = 4, iteration = 360/496, loss = 0.29026
epoch = 4, iteration = 370/496, loss = 1.67818
epoch = 4, iteration = 380/496, loss = 0.43088
epoch = 4, iteration = 390/496, loss = 1.10962
epoch = 4, iteration = 400/496, loss = 0.81793
epoch = 4, it

epoch = 7, iteration = 410/496, loss = 1.34879
epoch = 7, iteration = 420/496, loss = 0.31961
epoch = 7, iteration = 430/496, loss = 0.49232
epoch = 7, iteration = 440/496, loss = 0.52003
epoch = 7, iteration = 450/496, loss = 0.52605
epoch = 7, iteration = 460/496, loss = 0.69232
epoch = 7, iteration = 470/496, loss = 0.97181
epoch = 7, iteration = 480/496, loss = 0.43766
epoch = 7, iteration = 490/496, loss = 0.92625
Epoch 7 finished! Loss: 0.8030658328774. Accuracy: 0.6421370967741935
Starting epoch 8/10.
epoch = 8, iteration = 0/496, loss = 0.88469
epoch = 8, iteration = 10/496, loss = 0.38051
epoch = 8, iteration = 20/496, loss = 0.48067
epoch = 8, iteration = 30/496, loss = 0.70470
epoch = 8, iteration = 40/496, loss = 0.58672
epoch = 8, iteration = 50/496, loss = 0.77218
epoch = 8, iteration = 60/496, loss = 0.22882
epoch = 8, iteration = 70/496, loss = 0.54990
epoch = 8, iteration = 80/496, loss = 0.36306
epoch = 8, iteration = 90/496, loss = 0.46654
epoch = 8, iteration = 100/

### Evaluate Model

In [6]:
model.eval()

true_labels = []
pred_labels = []

for i, sample_batch in enumerate(test_loader):
    
    sequences = sample_batch['sequence']
    true_batch = sample_batch['label']
    
    outputs = model(sequences)
    _, pred_batch = torch.max(outputs, 1)
    
    true_labels.append(true_batch.detach().numpy())
    pred_labels.append(pred_batch.detach().numpy())

true_labels = np.concatenate(true_labels).squeeze()
pred_labels = np.concatenate(pred_labels)

In [7]:
acc = (true_labels==pred_labels).sum()/len(true_labels)
cm = confusion_matrix(true_labels, pred_labels)
cr = classification_report(true_labels, pred_labels)
print(acc)
print(cm)
print(cr)

0.9045454545454545
[[ 34   2   0   0]
 [  0  32   4   0]
 [  0   0  20  15]
 [  0   0   0 113]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        36
           1       0.94      0.89      0.91        36
           2       0.83      0.57      0.68        35
           3       0.88      1.00      0.94       113

    accuracy                           0.90       220
   macro avg       0.91      0.85      0.88       220
weighted avg       0.90      0.90      0.90       220



### Attention model testing

In [8]:
seq_size = 30
input_size = 10
hidden_size = 15
batch_size = 4
output_size = 3

lstm = nn.LSTM(input_size, hidden_size, num_layers=2) 
concat = nn.Linear(hidden_size * 2, hidden_size)
logits = nn.Linear(hidden_size, output_size)

In [9]:
x = torch.randn(seq_size, batch_size, input_size) 
out, (h,c) = lstm(x)

In [10]:
hidden = h[-1].squeeze(0)
output = out.permute(1,0,2)

print(hidden.size())
print(output.size())

score = torch.bmm(output, hidden.unsqueeze(2))
attn_weights = F.softmax(score, 1) # eq.7 
context = torch.bmm(output.transpose(1, 2), attn_weights).squeeze(2)

torch.Size([4, 15])
torch.Size([4, 30, 15])


In [11]:
concat_input = torch.cat((hidden, context), 1)
concat_output = torch.tanh(concat(concat_input)) # eq. 5
output = logits(concat_output) # eq. 6

In [12]:
out.size()

torch.Size([30, 4, 15])

In [13]:
hidden.unsqueeze(2).size()

torch.Size([4, 15, 1])

In [14]:
out.size()

torch.Size([30, 4, 15])